#### In this demo we will create a star schema from the udacity tutorial

In [1]:
import psycopg2

##### Connect to the db

In [2]:
conn=psycopg2.connect("host=127.0.0.1 dbname=data_engineering user=postgres password=macbook")


In [3]:
conn.set_session(autocommit=True)
cur=conn.cursor()

### Let's imagine we work at an online Music Store. There will be many tables in our database but let's just focus on 4 tables around customer purchases. 

`Table Name: customer_transactions
column: Customer Id
column: Store Id
column: Spent`

`Table Name: Customer
column: Customer Id
column: Name
column: Rewards`

`Table Name: store
column: Store Id
column: State`

`Table Name: items_purchased
column: customer id
column: Item Name`


#### From this representation we can already start to see the makings of a "STAR". We have one fact table (the center of the star) and 3  dimension tables that are coming from it.

In [5]:
cur.execute("create table  if not exists customer_transactions(customer_id int,store_id int,spent int)")


In [6]:
query="insert into customer_transactions(customer_id,store_id,spent)values(%s,%s,%s)"

In [7]:
try:
    cur.execute(query,(1,1,20.50))
    cur.execute(query,(2,1,35.21))
except psycopg2.Error as e:
    print(e)

##### Create other table(dimension) 

In [10]:
cur.execute("create table if not exists item_purchased(customer_id int,item_number int,item_name varchar)")
query="INSERT INTO item_purchased (customer_id, item_number, item_name) \
                 VALUES (%s, %s, %s)"

In [11]:
cur.execute(query,(1,1,"Rubber soul"))
cur.execute(query,(2,3,"Let it go"))


In [14]:
cur.execute("create table if not exists store(store_id int,state varchar)")
query="insert into store(store_id,state)values(%s,%s)"

In [15]:
cur.execute(query,(1,"CA"))
cur.execute(query,(2,"WA"))

In [16]:
### customer table
cur.execute("create table if not exists customer(customer_id int,name varchar, rewards boolean)")

In [19]:
query="insert into customer(customer_id,name,rewards)values(%s,%s,%s)"
cur.execute(query,(1,'Amanda',True))
cur.execute(query,(2,'Toby',False))

**We can do a variety of queries on this data easily because of utilizing the fact/dimension and Star Schema**

* _Query 1_: Find all the customers that spent more than 30 dollars, who are they, what did they buy and if they are a rewards member

* _Query 2_: How much did Store 1 sell?

In [21]:
cur.execute("select * from customer join customer_transactions on customer_transactions.customer_id=customer.customer_id where customer_transactions.spent>30")

In [22]:
val=cur.fetchall()
for i in val:
    print(i)

(2, 'Toby', False, 2, 1, 35)


In [24]:
cur.execute("select sum(spent) from customer_transactions group by store_id ")
val=cur.fetchall()
for i in val:
    print(i)

(56,)


#### Drop all the table 

In [26]:
try: 
    cur.execute("DROP table customer_transactions")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table item_purchased")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table customer")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)
try: 
    cur.execute("DROP table store")
except psycopg2.Error as e: 
    print("Error: Dropping table")
    print (e)

Error: Dropping table
table "customer_transactions" does not exist

Error: Dropping table
table "customer" does not exist

Error: Dropping table
table "store" does not exist



### Close the connections

In [27]:
cur.close()
conn.close()